<a href="https://colab.research.google.com/github/mohanrajmit/Transfer-Learning/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from pathlib import Path
import numpy as np
import joblib

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [2]:
!git clone https://github.com/mohanrajmit/Transfer-Learning.git

fatal: destination path 'Transfer-Learning' already exists and is not an empty directory.


In [3]:
!ls

bird_feature_classifier_model_vgg.h5  sample_data  Transfer-Learning


In [4]:
# Load the training data set by looping over every image file
for image_file in Path("Transfer-Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224, 224))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [5]:
print(np.array(x_train).shape)

(2000, 224, 224, 3)


In [6]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = imagenet_utils.preprocess_input(x_train)



In [7]:

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))



In [8]:
feature_extractor.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
#feature_extractor_resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))

In [10]:
feature_extractor.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
#feature_extractor_resnet.summary()

In [12]:
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)



63/63 [==============================] - 15s 150ms/step


In [13]:
features_x.shape

(2000, 7, 7, 512)

In [14]:
#features_resnet = feature_extractor_resnet.predict(x_train)

In [15]:
#features_resnet.shape

In [16]:
Y_train = np.array(y_train)
print(x_train.shape[1:])
print(Y_train.shape)

(224, 224, 3)
(2000,)


In [17]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [18]:
# Create a model_vgg and add layers
model_vgg = Sequential()

# Add layers to our model_vgg
model_vgg.add(Flatten(input_shape=features_x.shape[1:]))
model_vgg.add(Dense(128, activation='relu'))
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(1, activation='sigmoid'))

# Compile the model_vgg
model_vgg.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [19]:
model_vgg.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 128)               3211392   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3211521 (12.25 MB)
Trainable params: 3211521 (12.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
# Create a model_vgg and add layers
'''model_resnet = Sequential()

# Add layers to our model_vgg
model_resnet.add(Flatten(input_shape=features_resnet.shape[1:]))
model_resnet.add(Dense(128, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(1, activation='sigmoid'))

# Compile the model_vgg
model_resnet.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])'''


"model_resnet = Sequential()\n\n# Add layers to our model_vgg\nmodel_resnet.add(Flatten(input_shape=features_resnet.shape[1:]))\nmodel_resnet.add(Dense(128, activation='relu'))\nmodel_resnet.add(Dropout(0.5))\nmodel_resnet.add(Dense(1, activation='sigmoid'))\n\n# Compile the model_vgg\nmodel_resnet.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])"

In [22]:
# Train the model_vgg
model_vgg.fit(features_x,Y_train,epochs=200, verbose=2)

# Save the trained model_vgg to a file so we can use it to make predictions later
model_vgg.save("bird_feature_classifier_model_vgg.h5")

Epoch 1/200
63/63 - 5s - loss: 1.8159 - accuracy: 0.6975 - 5s/epoch - 86ms/step
Epoch 2/200
63/63 - 0s - loss: 0.3856 - accuracy: 0.8260 - 317ms/epoch - 5ms/step
Epoch 3/200
63/63 - 0s - loss: 0.3024 - accuracy: 0.8750 - 334ms/epoch - 5ms/step
Epoch 4/200
63/63 - 0s - loss: 0.2464 - accuracy: 0.8825 - 324ms/epoch - 5ms/step
Epoch 5/200
63/63 - 0s - loss: 0.2065 - accuracy: 0.9060 - 321ms/epoch - 5ms/step
Epoch 6/200
63/63 - 0s - loss: 0.1686 - accuracy: 0.9165 - 311ms/epoch - 5ms/step
Epoch 7/200
63/63 - 0s - loss: 0.1503 - accuracy: 0.9320 - 322ms/epoch - 5ms/step
Epoch 8/200
63/63 - 0s - loss: 0.1294 - accuracy: 0.9480 - 338ms/epoch - 5ms/step
Epoch 9/200
63/63 - 0s - loss: 0.1211 - accuracy: 0.9490 - 274ms/epoch - 4ms/step
Epoch 10/200
63/63 - 0s - loss: 0.1257 - accuracy: 0.9360 - 256ms/epoch - 4ms/step
Epoch 11/200
63/63 - 0s - loss: 0.1120 - accuracy: 0.9475 - 257ms/epoch - 4ms/step
Epoch 12/200
63/63 - 0s - loss: 0.1059 - accuracy: 0.9475 - 260ms/epoch - 4ms/step
Epoch 13/200
63

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Train the model_vgg
'''model_resnet.fit(features_resnet,Y_train,epochs=10,shuffle=True, verbose=2)

# Save the trained model_vgg to a file so we can use it to make predictions later
model_resnet.save("bird_feature_classifier_model_resnet.h5")'''

In [23]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [24]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224,224,3))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = imagenet_utils.preprocess_input(x_test)

In [25]:
x_test.shape

(200, 224, 224, 3)

In [26]:
# Load our trained classifier model_vgg
model_vgg = load_model("bird_feature_classifier_model_vgg.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model_vgg
predictions = model_vgg.predict(features_x)

# If the model_vgg is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model_vgg makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

7/7 [==============================] - 0s 5ms/step
True Positives: 89
True Negatives: 87
False Positives: 13
False Negatives: 11
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       100
           1       0.87      0.89      0.88       100

    accuracy                           0.88       200
   macro avg       0.88      0.88      0.88       200
weighted avg       0.88      0.88      0.88       200



In [ ]:
# Load our trained classifier model_vgg
'''model_resnet = load_model("bird_feature_classifier_model_resnet.h5")

# Extract features for each image (all in one pass)
features_resnet = feature_extractor_resnet.predict(x_test)

# Given the extracted features, make a final prediction using our own model_vgg
predictions = model_resnet.predict(features_resnet)

# If the model_vgg is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model_vgg makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)'''